In [253]:
import json
import numpy as np
import pandas as pd
import re

In [254]:
from kafka import KafkaConsumer
from pymongo import MongoClient
import json

# Kết nối đến MongoDB
mongo_client = MongoClient('mongodb://localhost:27017/')
db = mongo_client['dtb_data']
collection = db['clt_data']

In [255]:
# Lấy ra toàn bộ dữ liệu từ MongoDB
data = list(collection.find())

In [256]:
import pandas as pd

# Chuyển đổi danh sách dữ liệu thành DataFrame
df = pd.DataFrame(data)

In [257]:
df


,_id,product_id,name,price,special_price,total_count,manufacturer,operating_system,os_version,battery,...,storage,product_weight,mobile_tan_so_quet,loai_mang,mobile_cong_sac,mobile_cam_bien_van_tay,warranty_information,average_rating,key_selling_points,promotion_information
0,67345f600b1913b3765e2678,59258,iPhone 16 Pro Max 256GB | Chính hãng VN/A,34990000.0,34590000.0,36,Apple,iOS,iOS 18,N/A,...,256 GB,227 gram,N/A,5G,USB Type-C,N/A,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
1,67345f600b1913b3765e2679,59294,Samsung Galaxy S24 Ultra 12GB 256GB,33990000.0,29990000.0,31,Samsung,Android,"Android 14, One UI 6.1","5,000mAh",...,256 GB,232g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.9,Trải nghiệm đỉnh cao với hiệu năng mạnh mẽ từ ...,"Trả góp 0% đến 12 tháng, 0đ trả trước qua Sams..."
2,67345f600b1913b3765e267a,74613,Samsung Galaxy S24 FE 5G 8GB 128GB,16990000.0,0.0,2,Samsung,Android,Android 14,4700 mAh,...,128 GB,213 g,N/A,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.5,Màn hình Dynamic AMOLED 120Hz - Mang lại trải ...,Giảm 1.000.000đ qua QR bank và2 kmkhác
3,67345f600b1913b3765e267b,82082,Xiaomi Redmi 14C 4GB 128GB,3290000.0,3190000.0,8,Xiaomi,N/A,Android 13,5160mAh,...,128 GB,211g (Đen)<br>204g (Xanh dương),N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,"Màn hình 6,88 inch, lớn nhất trên dòng Redmi c...",Giảm 200.000 khi mua kèm Đồng hồ Redmi Watch 5...
4,67345f600b1913b3765e267c,80473,Samsung Galaxy M55 (12GB 256GB),11190000.0,10190000.0,3,Samsung,Android,Android 14,5000 mAh,...,256 GB,180 g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,5.0,Trải nghiệm hình ảnh sống động và sắc nét với ...,"PMH200,000 ₫Trả góp 0% đến 12 tháng, 0đ trả tr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,67345f600b1913b3765e2af8,31474,Huawei Nova 9,0.0,0.0,0,Huawei,Android,"Android 10, EMUI 11, không có Dịch vụ của Goog...",Li-Po 4000 mAh,...,256 GB,184 g,N/A,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành chính hãng 12 tháng tại trung tâm bảo...,0.0,N/A,N/A
1153,67345f600b1913b3765e2af9,72335,Điện thoại Tecno Camon 30,0.0,0.0,0,Hãng khác,N/A,N/A,6.000 mAh,...,512 GB,N/A,N/A,N/A,N/A,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,N/A,N/A
1154,67345f600b1913b3765e2afa,35666,Xiaomi Mi Mix Flip,0.0,0.0,0,Xiaomi,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,0.0,N/A,N/A
1155,67345f600b1913b3765e2afb,45902,Huawei Mate X3,0.0,0.0,0,Huawei,N/A,EMUI 13.1,4800 mAh,...,512 GB,239 g/241g,120Hz,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,"Vi xử lý Snapdragon 8 Plus Gen 1 mạnh mẽ, bứt ...",N/A


In [258]:
df = df.drop('_id', axis=1)

In [259]:
df

,product_id,name,price,special_price,total_count,manufacturer,operating_system,os_version,battery,chipset,...,storage,product_weight,mobile_tan_so_quet,loai_mang,mobile_cong_sac,mobile_cam_bien_van_tay,warranty_information,average_rating,key_selling_points,promotion_information
0,59258,iPhone 16 Pro Max 256GB | Chính hãng VN/A,34990000.0,34590000.0,36,Apple,iOS,iOS 18,N/A,Apple A18 Pro,...,256 GB,227 gram,N/A,5G,USB Type-C,N/A,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
1,59294,Samsung Galaxy S24 Ultra 12GB 256GB,33990000.0,29990000.0,31,Samsung,Android,"Android 14, One UI 6.1","5,000mAh",Snapdragon 8 Gen 3 For Galaxy,...,256 GB,232g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.9,Trải nghiệm đỉnh cao với hiệu năng mạnh mẽ từ ...,"Trả góp 0% đến 12 tháng, 0đ trả trước qua Sams..."
2,74613,Samsung Galaxy S24 FE 5G 8GB 128GB,16990000.0,0.0,2,Samsung,Android,Android 14,4700 mAh,Exynos 2400e (4nm),...,128 GB,213 g,N/A,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.5,Màn hình Dynamic AMOLED 120Hz - Mang lại trải ...,Giảm 1.000.000đ qua QR bank và2 kmkhác
3,82082,Xiaomi Redmi 14C 4GB 128GB,3290000.0,3190000.0,8,Xiaomi,N/A,Android 13,5160mAh,MediaTek Helio G81-Ultra,...,128 GB,211g (Đen)<br>204g (Xanh dương),N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,"Màn hình 6,88 inch, lớn nhất trên dòng Redmi c...",Giảm 200.000 khi mua kèm Đồng hồ Redmi Watch 5...
4,80473,Samsung Galaxy M55 (12GB 256GB),11190000.0,10190000.0,3,Samsung,Android,Android 14,5000 mAh,Qualcomm Snapdragon 7 Gen 1 (4 nm),...,256 GB,180 g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,5.0,Trải nghiệm hình ảnh sống động và sắc nét với ...,"PMH200,000 ₫Trả góp 0% đến 12 tháng, 0đ trả tr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,31474,Huawei Nova 9,0.0,0.0,0,Huawei,Android,"Android 10, EMUI 11, không có Dịch vụ của Goog...",Li-Po 4000 mAh,Kirin 985 5G (7 nm),...,256 GB,184 g,N/A,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành chính hãng 12 tháng tại trung tâm bảo...,0.0,N/A,N/A
1153,72335,Điện thoại Tecno Camon 30,0.0,0.0,0,Hãng khác,N/A,N/A,6.000 mAh,MediaTek Dimensity 1000,...,512 GB,N/A,N/A,N/A,N/A,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,N/A,N/A
1154,35666,Xiaomi Mi Mix Flip,0.0,0.0,0,Xiaomi,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,0.0,N/A,N/A
1155,45902,Huawei Mate X3,0.0,0.0,0,Huawei,N/A,EMUI 13.1,4800 mAh,Snapdragon 8+ Gen 1 4G,...,512 GB,239 g/241g,120Hz,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,"Vi xử lý Snapdragon 8 Plus Gen 1 mạnh mẽ, bứt ...",N/A


In [260]:
# Xóa các dòng trùng lặp
df = df.drop_duplicates()

# Hiển thị DataFrame sau khi xóa
print("\nDataFrame sau khi xóa dòng trùng:")
print(df)


DataFrame sau khi xóa dòng trùng:
      product_id                                       name       price  \
0          59258  iPhone 16 Pro Max 256GB | Chính hãng VN/A  34990000.0   
1          59294        Samsung Galaxy S24 Ultra 12GB 256GB  33990000.0   
2          74613         Samsung Galaxy S24 FE 5G 8GB 128GB  16990000.0   
3          82082                 Xiaomi Redmi 14C 4GB 128GB   3290000.0   
4          80473            Samsung Galaxy M55 (12GB 256GB)  11190000.0   
...          ...                                        ...         ...   
1150       89503                             Mobell M139 4G         0.0   
1151       84487                            Google Pixel 8A         0.0   
1154       35666                         Xiaomi Mi Mix Flip         0.0   
1155       45902                             Huawei Mate X3         0.0   
1156       19102                   Xiaomi Redmi Note 8 32GB   3990000.0   

      special_price  total_count manufacturer operating_system  

In [261]:
df

,product_id,name,price,special_price,total_count,manufacturer,operating_system,os_version,battery,chipset,...,storage,product_weight,mobile_tan_so_quet,loai_mang,mobile_cong_sac,mobile_cam_bien_van_tay,warranty_information,average_rating,key_selling_points,promotion_information
0,59258,iPhone 16 Pro Max 256GB | Chính hãng VN/A,34990000.0,34590000.0,36,Apple,iOS,iOS 18,N/A,Apple A18 Pro,...,256 GB,227 gram,N/A,5G,USB Type-C,N/A,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
1,59294,Samsung Galaxy S24 Ultra 12GB 256GB,33990000.0,29990000.0,31,Samsung,Android,"Android 14, One UI 6.1","5,000mAh",Snapdragon 8 Gen 3 For Galaxy,...,256 GB,232g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.9,Trải nghiệm đỉnh cao với hiệu năng mạnh mẽ từ ...,"Trả góp 0% đến 12 tháng, 0đ trả trước qua Sams..."
2,74613,Samsung Galaxy S24 FE 5G 8GB 128GB,16990000.0,0.0,2,Samsung,Android,Android 14,4700 mAh,Exynos 2400e (4nm),...,128 GB,213 g,N/A,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.5,Màn hình Dynamic AMOLED 120Hz - Mang lại trải ...,Giảm 1.000.000đ qua QR bank và2 kmkhác
3,82082,Xiaomi Redmi 14C 4GB 128GB,3290000.0,3190000.0,8,Xiaomi,N/A,Android 13,5160mAh,MediaTek Helio G81-Ultra,...,128 GB,211g (Đen)<br>204g (Xanh dương),N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,"Màn hình 6,88 inch, lớn nhất trên dòng Redmi c...",Giảm 200.000 khi mua kèm Đồng hồ Redmi Watch 5...
4,80473,Samsung Galaxy M55 (12GB 256GB),11190000.0,10190000.0,3,Samsung,Android,Android 14,5000 mAh,Qualcomm Snapdragon 7 Gen 1 (4 nm),...,256 GB,180 g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,5.0,Trải nghiệm hình ảnh sống động và sắc nét với ...,"PMH200,000 ₫Trả góp 0% đến 12 tháng, 0đ trả tr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,89503,Mobell M139 4G,0.0,0.0,0,Hãng khác,N/A,N/A,1000 mAH,N/A,...,N/A,158 g,N/A,4G,N/A,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,Thiết kế các góc bo tròn mềm mại của sản phẩm ...,N/A
1151,84487,Google Pixel 8A,0.0,0.0,0,Hãng khác,Android,Android 14,4492 mAh,Google Tensor G3 (4 nm),...,256 MB,183g,N/A,N/A,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,Màn hình sống động mang đến trải nghiệm hình ả...,N/A
1154,35666,Xiaomi Mi Mix Flip,0.0,0.0,0,Xiaomi,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,0.0,N/A,N/A
1155,45902,Huawei Mate X3,0.0,0.0,0,Huawei,N/A,EMUI 13.1,4800 mAh,Snapdragon 8+ Gen 1 4G,...,512 GB,239 g/241g,120Hz,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,"Vi xử lý Snapdragon 8 Plus Gen 1 mạnh mẽ, bứt ...",N/A


In [262]:
# Làm sạch tên sản phẩm
df['name'] = df['name'].str.strip().str.title()
df

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3799598484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].str.strip().str.title()


,product_id,name,price,special_price,total_count,manufacturer,operating_system,os_version,battery,chipset,...,storage,product_weight,mobile_tan_so_quet,loai_mang,mobile_cong_sac,mobile_cam_bien_van_tay,warranty_information,average_rating,key_selling_points,promotion_information
0,59258,Iphone 16 Pro Max 256Gb | Chính Hãng Vn/A,34990000.0,34590000.0,36,Apple,iOS,iOS 18,N/A,Apple A18 Pro,...,256 GB,227 gram,N/A,5G,USB Type-C,N/A,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
1,59294,Samsung Galaxy S24 Ultra 12Gb 256Gb,33990000.0,29990000.0,31,Samsung,Android,"Android 14, One UI 6.1","5,000mAh",Snapdragon 8 Gen 3 For Galaxy,...,256 GB,232g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.9,Trải nghiệm đỉnh cao với hiệu năng mạnh mẽ từ ...,"Trả góp 0% đến 12 tháng, 0đ trả trước qua Sams..."
2,74613,Samsung Galaxy S24 Fe 5G 8Gb 128Gb,16990000.0,0.0,2,Samsung,Android,Android 14,4700 mAh,Exynos 2400e (4nm),...,128 GB,213 g,N/A,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,4.5,Màn hình Dynamic AMOLED 120Hz - Mang lại trải ...,Giảm 1.000.000đ qua QR bank và2 kmkhác
3,82082,Xiaomi Redmi 14C 4Gb 128Gb,3290000.0,3190000.0,8,Xiaomi,N/A,Android 13,5160mAh,MediaTek Helio G81-Ultra,...,128 GB,211g (Đen)<br>204g (Xanh dương),N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,"Màn hình 6,88 inch, lớn nhất trên dòng Redmi c...",Giảm 200.000 khi mua kèm Đồng hồ Redmi Watch 5...
4,80473,Samsung Galaxy M55 (12Gb 256Gb),11190000.0,10190000.0,3,Samsung,Android,Android 14,5000 mAh,Qualcomm Snapdragon 7 Gen 1 (4 nm),...,256 GB,180 g,120Hz,5G,USB Type-C,Cảm biến vân tay trong màn hình,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,5.0,Trải nghiệm hình ảnh sống động và sắc nét với ...,"PMH200,000 ₫Trả góp 0% đến 12 tháng, 0đ trả tr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,89503,Mobell M139 4G,0.0,0.0,0,Hãng khác,N/A,N/A,1000 mAH,N/A,...,N/A,158 g,N/A,4G,N/A,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,Thiết kế các góc bo tròn mềm mại của sản phẩm ...,N/A
1151,84487,Google Pixel 8A,0.0,0.0,0,Hãng khác,Android,Android 14,4492 mAh,Google Tensor G3 (4 nm),...,256 MB,183g,N/A,N/A,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,Màn hình sống động mang đến trải nghiệm hình ả...,N/A
1154,35666,Xiaomi Mi Mix Flip,0.0,0.0,0,Xiaomi,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,0.0,N/A,N/A
1155,45902,Huawei Mate X3,0.0,0.0,0,Huawei,N/A,EMUI 13.1,4800 mAh,Snapdragon 8+ Gen 1 4G,...,512 GB,239 g/241g,120Hz,5G,USB Type-C,N/A,Bảo hành 12 tháng tại trung tâm bảo hành Chính...,0.0,"Vi xử lý Snapdragon 8 Plus Gen 1 mạnh mẽ, bứt ...",N/A


In [263]:
df.isna().sum()

product_id                 0
name                       0
price                      0
special_price              0
total_count                0
manufacturer               0
operating_system           0
os_version                 0
battery                    0
chipset                    0
display_size               0
display_resolution         0
mobile_type_of_display     0
memory_internal            0
storage                    0
product_weight             0
mobile_tan_so_quet         0
loai_mang                  0
mobile_cong_sac            0
mobile_cam_bien_van_tay    0
warranty_information       0
average_rating             0
key_selling_points         0
promotion_information      0
dtype: int64

In [264]:
#price
# Thay thế giá trị 0 bằng NaN
df['price'] = df['price'].replace(0, np.nan)

# Tính giá trị trung vị của cột price, loại bỏ NaN
median_price = df['price'].median()

# Thay thế giá trị NaN bằng giá trị trung vị
df['price'].fillna(median_price, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2723101989.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].replace(0, np.nan)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2723101989.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(

In [265]:
#special_price
# Thay thế giá trị 0 bằng 'isnull'
df['special_price'] = df['special_price'].replace(0, np.nan)
# Thay thế giá trị NaN trong special_price bằng giá trị tương ứng từ price 
df['special_price'] = df['special_price'].fillna(df['price'])

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\297360362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['special_price'] = df['special_price'].replace(0, np.nan)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\297360362.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['special_price'] = df['special_price'].fillna(df['price'])


In [266]:
df['operating_system']

0           iOS
1       Android
2       Android
3           N/A
4       Android
         ...   
1150        N/A
1151    Android
1154        N/A
1155        N/A
1156    Android
Name: operating_system, Length: 1037, dtype: object

In [267]:
na_count = df['operating_system'].value_counts().get('N/A', 0)
na_count

np.int64(335)

In [268]:
#operating_system
# Tìm giá trị xuất hiện nhiều nhất trong cột
most_common_value = df['operating_system'].mode()[0]  # mode() trả về một Series

# Thay thế giá trị "N/A" bằng giá trị xuất hiện nhiều nhất
df['operating_system'] = df['operating_system'].replace('N/A', most_common_value)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1031897609.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['operating_system'] = df['operating_system'].replace('N/A', most_common_value)


In [269]:
#os_version
na_count = df['os_version'].value_counts().get('N/A', 0)
na_count

np.int64(282)

In [270]:
# Thay thế giá trị "N/A" bằng np.nan
df['os_version'] = df['os_version'].replace('N/A', np.nan)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\4266860671.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['os_version'] = df['os_version'].replace('N/A', np.nan)


In [271]:
# Tìm giá trị xuất hiện nhiều nhất trong cột
most_common_value = df['os_version'].mode()[0]  # mode() trả về một Series
most_common_value



'Android 13'

In [272]:
# Thay thế giá trị "N/A" bằng giá trị xuất hiện nhiều nhất
df['os_version'] = df['os_version'].fillna(most_common_value)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1614895537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['os_version'] = df['os_version'].fillna(most_common_value)


In [273]:
# Lọc dữ liệu cột battery, chỉ lấy đoạn dữ liệu có chứa 'mAh' sau đó mới phân tích, loại bỏ đơn vị 
df['battery'] = df['battery'].apply(lambda x: x if isinstance(x, str) and 'mAh' in x else '0 mAh')
df['battery'] = df['battery'].apply(lambda x: x.split('mAh')[0] if isinstance(x, str) else x)

def clean_battery(battery):
    if pd.isna(battery):
        return 0
    # Loại bỏ đơn vị mAh
    battery = re.sub(r'[^\d,]', '', str(battery))
    # Chuyển đổi giá trị từ dạng 5,232 thành 5232
    battery = battery.replace(',', '')
    battery = battery.replace('.', '')
    return int(battery) if battery else 0

df['battery'] = df['battery'].apply(clean_battery)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3922293630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['battery'] = df['battery'].apply(lambda x: x if isinstance(x, str) and 'mAh' in x else '0 mAh')
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3922293630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['battery'] = df['battery'].apply(lambda x: x.split('mAh')[0] if isinstance(x, str) else x)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3922293630.py:15: SettingWithCopyWarning: 
A v

In [274]:
# Thay thế giá trị 'N/A' bằng np.nan
df['battery'] = df['battery'].replace('N/A', np.nan)

# Hàm để trích xuất giá trị số từ chuỗi
def extract_battery_value(battery_str):
    # Biểu thức chính quy để tìm giá trị số gần nhất với 'mAh' hoặc 'mah'
    match = re.search(r'(\d[\d,.]*)\s*[mM][aA][hH]', battery_str)
    if match:
        # Làm sạch và chuyển đổi thành số
        value = match.group(1).replace(',', '').replace('.', '')  # Xóa dấu phẩy và dấu chấm
        try:
            return int(value)  # Trả về số nguyên
        except ValueError:
            return np.nan
    return np.nan  # Nếu không tìm thấy, trả về np.nan

# Áp dụng hàm để lấy giá trị số từ chuỗi
df['battery'] = df['battery'].apply(lambda x: extract_battery_value(x) if isinstance(x, str) else x)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1654576316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['battery'] = df['battery'].replace('N/A', np.nan)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1654576316.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['battery'] = df['battery'].apply(lambda x: extract_battery_value(x) if isinstance(x, str) else x)


In [275]:
na_count = df['battery'].value_counts().get(0)
na_count

np.int64(195)

In [276]:
# Tìm giá trị nhỏ nhất trong cột battery, loại trừ giá trị 0
min_value = df['battery'][df['battery'] > 0].min()

# Thay thế các giá trị 0 bằng giá trị nhỏ nhất
df['battery'] = df['battery'].replace(0, min_value)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2856140676.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['battery'] = df['battery'].replace(0, min_value)


In [277]:
#chipset
na_count = df['chipset'].value_counts().get('N/A',0)
na_count

np.int64(165)

In [278]:
df['chipset'] = df['chipset'].replace('N/A', 'Đang cập nhật')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2956536347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chipset'] = df['chipset'].replace('N/A', 'Đang cập nhật')


In [279]:
# display_size
na_count = df['display_size'].value_counts().get('N/A',0)
na_count

np.int64(172)

In [280]:

df['display_size'] = df['display_size'].apply(lambda x: re.sub(r'[^\d.]', '', str(x)) if isinstance(x, str) else '0').replace('', '0').astype(float)
min_display_size = df['display_size'][df['display_size'] != 0].min()
df['display_size'] = df['display_size'].apply(lambda x: min_display_size if x == 0 else x)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\879978747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_size'] = df['display_size'].apply(lambda x: re.sub(r'[^\d.]', '', str(x)) if isinstance(x, str) else '0').replace('', '0').astype(float)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\879978747.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_size'] = df['display_size'].apply(lambda x: min_display_size if x == 0 else x)


In [281]:
#display_resolution
na_count = df['display_resolution'].value_counts().get('N/A',0)
na_count

np.int64(335)

In [282]:
# Hàm để làm sạch dữ liệu
def clean_resolution(resolution):
    # Sử dụng regex để tìm kiếm định dạng số x số
    match = re.search(r'(\d+)\s*x\s*(\d+)', resolution)
    if match:
        return f"{match.group(1)} x {match.group(2)}"
    else:
        return None  # Hoặc có thể trả về một giá trị hợp lệ nào đó

# Áp dụng hàm làm sạch lên cột display_resolution
df['display_resolution'] = df['display_resolution'].apply(clean_resolution)


C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3843076270.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_resolution'] = df['display_resolution'].apply(clean_resolution)


In [283]:
# Tìm giá trị xuất hiện nhiều nhất trong cột
most_common_value = df['display_resolution'].mode()[0]  # mode() trả về một Series
most_common_value
df['display_resolution'] = df['display_resolution'].fillna(most_common_value)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2188583441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['display_resolution'] = df['display_resolution'].fillna(most_common_value)


In [284]:
na_count = df['mobile_type_of_display'].value_counts().get('N/A',0)
na_count

np.int64(264)

In [285]:
df['mobile_type_of_display'] = df['mobile_type_of_display'].replace('N/A', 'Đang cập nhật')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2746000373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mobile_type_of_display'] = df['mobile_type_of_display'].replace('N/A', 'Đang cập nhật')


In [286]:
#memory_internal   
# Hàm để trích xuất số từ đầu chuỗi
def extract_memory(value):
    # Sử dụng regex để tìm số ở bắt đầu chuỗi và trả về số nguyên
    match = re.match(r'(\d+)', value)
    if match:
        return int(match.group(1))  # Trả về số nguyên
    return None  # Trả về None nếu không tìm thấy số

# Áp dụng hàm trích xuất lên cột memory_internal
df['memory_internal'] = df['memory_internal'].apply(extract_memory)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\4273655705.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['memory_internal'] = df['memory_internal'].apply(extract_memory)


In [287]:
# Tính trung vị của cột extracted_memory
median_value = df['memory_internal'].median()

# Thay thế giá trị np.nan bằng trung vị
df['memory_internal'].fillna(median_value, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1578203512.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['memory_internal'].fillna(median_value, inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1578203512.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['memory_internal'].fillna(median_value, inplace=True)


In [288]:
# Hàm để chuyển đổi giá trị lưu trữ thành GB
def convert_storage(value):
    if 'TB' in value:
        # Chuyển đổi TB thành GB
        tb_value = float(re.match(r'(\d+(\.\d+)?)', value).group(1))
        return tb_value * 1024  # 1 TB = 1024 GB
    elif 'GB' in value:
        # Trả lại giá trị GB đã cho
        return float(re.match(r'(\d+(\.\d+)?)', value).group(1))
    elif 'MB' in value:
        # Chuyển đổi MB thành GB
        mb_value = float(re.match(r'(\d+(\.\d+)?)', value).group(1))
        return mb_value / 1024  # 1 GB = 1024 MB
    return np.nan  # Trả về np.nan nếu không tìm thấy số

# Áp dụng hàm chuyển đổi lên cột storage
df['storage'] = df['storage'].apply(convert_storage)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2664018487.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['storage'] = df['storage'].apply(convert_storage)


In [289]:
# Tính trung vị của cột storage_gb
median_value = df['storage'].median()

# Thay thế giá trị np.nan bằng trung vị
df['storage'].fillna(median_value, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1784078327.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['storage'].fillna(median_value, inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1784078327.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['storage'].fillna(median_value, inplace=True)


In [290]:
# Hàm để lấy số đầu tiên trong chuỗi
def extract_first_number(weight_string):
    # Sử dụng regex để tìm số đầu tiên
    match = re.search(r'(\d+(\.\d+)?)', weight_string)
    if match:
        return float(match.group(1))  # Chuyển đổi thành float nếu có số
    return np.nan  # Trả về np.nan nếu không tìm thấy

# Áp dụng hàm lên cột product_weight
df['product_weight'] = df['product_weight'].apply(extract_first_number)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2616654913.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_weight'] = df['product_weight'].apply(extract_first_number)


In [291]:
# Tính trung vị của cột storage_gb
median_value = df['product_weight'].median()

# Thay thế giá trị np.nan bằng trung vị
df['product_weight'].fillna(median_value, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2157143236.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['product_weight'].fillna(median_value, inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2157143236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product_weight'].fillna(median_value, inplace=True)


In [292]:
# Hàm để lấy số từ chuỗi
def extract_frequency(freq_string):
    match = re.search(r'(\d+)', freq_string)  # Tìm số đầu tiên
    if match:
        return int(match.group(1))  # Trả về số dưới dạng int
    return np.nan  # Trả về np.nan nếu không tìm thấy số

# Áp dụng hàm lên cột mobile_tan_so_quet
df['mobile_tan_so_quet'] = df['mobile_tan_so_quet'].apply(extract_frequency)
# Tính trung vị của cột storage_gb
median_value = df['mobile_tan_so_quet'].median()

# Thay thế giá trị np.nan bằng trung vị
df['mobile_tan_so_quet'].fillna(median_value, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2301502033.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mobile_tan_so_quet'] = df['mobile_tan_so_quet'].apply(extract_frequency)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2301502033.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the origina

In [293]:
na_count = df['loai_mang'].value_counts().get('N/A',0)
na_count

np.int64(342)

In [294]:
# Tạo một cột mới để lấy giá trị chỉ số (bỏ chữ G)
df['loai_mang'] = df['loai_mang'].str.extract('(\d+)')  # Trích xuất số

# Chuyển đổi kiểu dữ liệu sang int, bỏ qua NaN
df['loai_mang'] = df['loai_mang'].astype(float)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1411150277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loai_mang'] = df['loai_mang'].str.extract('(\d+)')  # Trích xuất số
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1411150277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loai_mang'] = df['loai_mang'].astype(float)


In [295]:
# Tính trung vị của cột 
median_value = df['loai_mang'].median()

# Thay thế giá trị np.nan bằng trung vị
df['loai_mang'].fillna(median_value, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3056784705.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['loai_mang'].fillna(median_value, inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3056784705.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loai_mang'].fillna(median_value, inplace=True)


In [296]:
na_count = df['mobile_cong_sac'].value_counts().get('N/A',0)
na_count

np.int64(372)

In [297]:
# Tìm giá trị xuất hiện nhiều nhất trong cột
most_common_value = df['mobile_cong_sac'].mode()[0]  # mode() trả về một Series

# Thay thế giá trị "N/A" bằng giá trị xuất hiện nhiều nhất
df['mobile_cong_sac'] = df['mobile_cong_sac'].replace('N/A', most_common_value)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2754149447.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mobile_cong_sac'] = df['mobile_cong_sac'].replace('N/A', most_common_value)


In [298]:
na_count = df['mobile_cam_bien_van_tay'].value_counts().get('N/A',0)
na_count

np.int64(493)

In [299]:
df['mobile_cam_bien_van_tay'] = df['mobile_cam_bien_van_tay'].replace('N/A', 'không')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1093229083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mobile_cam_bien_van_tay'] = df['mobile_cam_bien_van_tay'].replace('N/A', 'không')


In [300]:
na_count = df['warranty_information'].value_counts().get('N/A',0)
na_count

np.int64(21)

In [301]:
df['warranty_information'] = df['warranty_information'].replace('N/A', 'Đang cập nhật')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\147600390.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['warranty_information'] = df['warranty_information'].replace('N/A', 'Đang cập nhật')


In [302]:
na_count = df['key_selling_points'].value_counts().get('N/A',0)
na_count

np.int64(688)

In [303]:
df['key_selling_points'] = df['key_selling_points'].replace('N/A', 'Đang cập nhật')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\3014898209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['key_selling_points'] = df['key_selling_points'].replace('N/A', 'Đang cập nhật')


In [304]:
na_count = df['promotion_information'].value_counts().get('N/A',0)
na_count

np.int64(860)

In [305]:
df['promotion_information'] = df['promotion_information'].replace('N/A', 'Đang cập nhật')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\706407484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['promotion_information'] = df['promotion_information'].replace('N/A', 'Đang cập nhật')


In [306]:
new_promotions = {
    29063: 'Hãng khác',
}

for product_id, new_value in new_promotions.items():
    df.loc[df['product_id'] == product_id, 'manufacturer'] = new_value

In [307]:
# Hàm để thêm 'Android' vào trước số đứng đầu
def prepend_android(version):
    # Kiểm tra xem dòng có bắt đầu bằng số không
    if re.match(r'^\d', version):
        return 'Android ' + version
    return version

# Áp dụng hàm vào cột 'os_version'
df['os_version'] = df['os_version'].apply(prepend_android)

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1226770588.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['os_version'] = df['os_version'].apply(prepend_android)


In [308]:
df['os_version'] = df['os_version'].replace('Android 5000 mAh', 'Android 12')

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2075248314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['os_version'] = df['os_version'].replace('Android 5000 mAh', 'Android 12')


In [309]:
# Thay thế tất cả giá trị 0 thành 5 trong cột 'average_rating'
df['average_rating'] = df['average_rating'].replace(0, 5)


C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\1888725515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average_rating'] = df['average_rating'].replace(0, 5)


In [310]:
# Giả sử chúng ta muốn sửa đổi promotion_information cho productid 102 và 103
new_promotions = {
    44939: 'đang cập nhật',
    50350: 'đang cập nhật',
    49374: 'đang cập nhật'
}

# Lặp qua dictionary để thay thế giá trị trong cột promotion_information
for product_id, new_value in new_promotions.items():
    df.loc[df['product_id'] == product_id, 'promotion_information'] = new_value

In [311]:
# Chuyển 0 thành np.nan
df['total_count'] = df['total_count'].replace(0, np.nan)

# Tính số lượng np.nan
nan_count = df['total_count'].isna().sum()

# Tạo giá trị ngẫu nhiên
random_values = np.random.randint(1, 31, size=nan_count)

# Thay thế np.nan bằng giá trị random
nan_indices = df['total_count'].index[df['total_count'].isna()]
df.loc[nan_indices, 'total_count'] = random_values

C:\Users\DELL\AppData\Local\Temp\ipykernel_17516\2476427878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_count'] = df['total_count'].replace(0, np.nan)


In [312]:
df.isna().sum()

product_id                 0
name                       0
price                      0
special_price              0
total_count                0
manufacturer               0
operating_system           0
os_version                 0
battery                    0
chipset                    0
display_size               0
display_resolution         0
mobile_type_of_display     0
memory_internal            0
storage                    0
product_weight             0
mobile_tan_so_quet         0
loai_mang                  0
mobile_cong_sac            0
mobile_cam_bien_van_tay    0
warranty_information       0
average_rating             0
key_selling_points         0
promotion_information      0
dtype: int64

In [313]:
df.dtypes

product_id                   int64
name                        object
price                      float64
special_price              float64
total_count                float64
manufacturer                object
operating_system            object
os_version                  object
battery                      int64
chipset                     object
display_size               float64
display_resolution          object
mobile_type_of_display      object
memory_internal            float64
storage                    float64
product_weight             float64
mobile_tan_so_quet         float64
loai_mang                  float64
mobile_cong_sac             object
mobile_cam_bien_van_tay     object
warranty_information        object
average_rating             float64
key_selling_points          object
promotion_information       object
dtype: object

In [314]:
df.to_csv('sanpham.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 1. Tạo bảng Manufacturers
manufacturers_df = df[['manufacturer']].drop_duplicates().reset_index(drop=True)
manufacturers_df.columns = ['manufacturer_name']
manufacturers_df['manufacturer_id'] = manufacturers_df.index + 1  # Gán id từ 1 trở đi

# 2. Tạo bảng Operating Systems
operating_systems_df = df[['operating_system', 'os_version']].drop_duplicates().reset_index(drop=True)
operating_systems_df.columns = ['os_name', 'os_version']
operating_systems_df['os_id'] = operating_systems_df.index + 1001  # Gán id từ 1001 trở đi

# 3. Tạo bảng Display
display_df = df[['display_size', 'display_resolution', 'mobile_type_of_display']].drop_duplicates().reset_index(drop=True)
display_df.columns = ['display_size', 'display_resolution', 'mobile_type_of_display']
display_df['display_id'] = display_df.index + 2001  # Gán id từ 2001 trở đi

# 4. Tạo bảng Products và giữ lại manufacturer_id và os_id, cũng như các cột cần thiết cho phép nối với display_df
products_df = df[['product_id', 'name', 'price', 'special_price', 
                   'total_count', 'manufacturer', 
                   'operating_system', 'battery', 'chipset', 
                   'memory_internal', 'storage', 
                   'product_weight', 
                   'mobile_tan_so_quet', 'loai_mang', 
                   'mobile_cong_sac', 'mobile_cam_bien_van_tay', 
                   'warranty_information', 'average_rating', 
                   'key_selling_points', 'promotion_information', 
                   'display_size', 'display_resolution']].copy()  # Thêm display_size và display_resolution vào đây

# Ghép manufacturer_id vào products_df
products_df = products_df.merge(manufacturers_df, 
                                 how='left', 
                                 left_on='manufacturer', 
                                 right_on='manufacturer_name')

# Ghép os_id vào products_df
products_df = products_df.merge(operating_systems_df[['os_name', 'os_version', 'os_id']], 
                                 how='left', 
                                 left_on='operating_system', 
                                 right_on='os_name')

# Ghép display_id vào products_df
products_df = products_df.merge(display_df, 
                                 how='left', 
                                 left_on=['display_size', 'display_resolution'], 
                                 right_on=['display_size', 'display_resolution'])

# Xóa cột không cần thiết
products_df = products_df[['product_id', 'name', 'price', 'special_price', 
                             'total_count', 'manufacturer_id', 'os_id', 
                             'display_id', 'battery', 'chipset', 
                             'memory_internal', 'storage', 
                             'product_weight', 
                             'mobile_tan_so_quet', 'loai_mang', 
                             'mobile_cong_sac', 'mobile_cam_bien_van_tay', 
                             'warranty_information', 'average_rating', 
                             'key_selling_points', 'promotion_information']]



Chovy


In [316]:
products_df

,product_id,name,price,special_price,total_count,manufacturer_id,os_id,display_id,battery,chipset,...,storage,product_weight,mobile_tan_so_quet,loai_mang,mobile_cong_sac,mobile_cam_bien_van_tay,warranty_information,average_rating,key_selling_points,promotion_information
0,59258,Iphone 16 Pro Max 256Gb | Chính Hãng Vn/A,34990000.0,34590000.0,36.0,1,1001,2001,1000,Apple A18 Pro,...,256.0,227.0,120.0,5.0,USB Type-C,không,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
1,59258,Iphone 16 Pro Max 256Gb | Chính Hãng Vn/A,34990000.0,34590000.0,36.0,1,1005,2001,1000,Apple A18 Pro,...,256.0,227.0,120.0,5.0,USB Type-C,không,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
2,59258,Iphone 16 Pro Max 256Gb | Chính Hãng Vn/A,34990000.0,34590000.0,36.0,1,1007,2001,1000,Apple A18 Pro,...,256.0,227.0,120.0,5.0,USB Type-C,không,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
3,59258,Iphone 16 Pro Max 256Gb | Chính Hãng Vn/A,34990000.0,34590000.0,36.0,1,1008,2001,1000,Apple A18 Pro,...,256.0,227.0,120.0,5.0,USB Type-C,không,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
4,59258,Iphone 16 Pro Max 256Gb | Chính Hãng Vn/A,34990000.0,34590000.0,36.0,1,1010,2001,1000,Apple A18 Pro,...,256.0,227.0,120.0,5.0,USB Type-C,không,1 ĐỔI 1 trong 30 ngày nếu có lỗi phần cứng nhà...,4.9,"Màn hình Super Retina XDR 6,9 inch lớn hơn có ...",Ưu đãi thanh toán giảm thêm đến 1.6 triệu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493614,19102,Xiaomi Redmi Note 8 32Gb,3990000.0,3350000.0,15.0,3,1155,2358,4000,Qualcomm Snapdragon 665 8 nhân,...,32.0,190.0,120.0,5.0,USB Type-C,không,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,Đang cập nhật,Đồng hồ XIAOMI REDMI WATCH 4 mua kèm điện thoạ...
493615,19102,Xiaomi Redmi Note 8 32Gb,3990000.0,3350000.0,15.0,3,1156,2328,4000,Qualcomm Snapdragon 665 8 nhân,...,32.0,190.0,120.0,5.0,USB Type-C,không,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,Đang cập nhật,Đồng hồ XIAOMI REDMI WATCH 4 mua kèm điện thoạ...
493616,19102,Xiaomi Redmi Note 8 32Gb,3990000.0,3350000.0,15.0,3,1156,2358,4000,Qualcomm Snapdragon 665 8 nhân,...,32.0,190.0,120.0,5.0,USB Type-C,không,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,Đang cập nhật,Đồng hồ XIAOMI REDMI WATCH 4 mua kèm điện thoạ...
493617,19102,Xiaomi Redmi Note 8 32Gb,3990000.0,3350000.0,15.0,3,1157,2328,4000,Qualcomm Snapdragon 665 8 nhân,...,32.0,190.0,120.0,5.0,USB Type-C,không,Bảo hành 18 tháng tại trung tâm bảo hành Chính...,4.9,Đang cập nhật,Đồng hồ XIAOMI REDMI WATCH 4 mua kèm điện thoạ...


In [317]:
# Xuất ra file CSV
manufacturers_df.to_csv('manufacturers.csv', index=False, encoding='utf-8-sig')
operating_systems_df.to_csv('operating_systems.csv', index=False, encoding='utf-8-sig')
display_df.to_csv('display.csv', index=False, encoding='utf-8-sig')
products_df.to_csv('products.csv', index=False, encoding='utf-8-sig')


In [318]:
import pandas as pd
from pymongo import MongoClient


# Tạo danh sách chứa tất cả DataFrame
dataframes = [df, manufacturers_df, operating_systems_df, display_df, products_df]
# Kết nối tới MongoDB
client = MongoClient('mongodb://localhost:27017/')  # Địa chỉ MongoDB
db = client['ptdl_datawebclean']  # Thay 'your_database_name' bằng tên database của bạn

# Tên collection cho từng DataFrame nếu cần
collection_names = ['telephone', 'manu', 'os', 'display', 'products']

# Lặp qua các DataFrame và chèn vào MongoDB
for df, collection_name in zip(dataframes, collection_names):
    collection = db[collection_name]  # Chọn collection tương ứng
    data_to_insert = df.to_dict(orient='records')  # Chuyển đổi DataFrame thành danh sách dict
    collection.insert_many(data_to_insert)  # Chèn dữ liệu vào MongoDB

print("All dataframes have been inserted into MongoDB successfully.")

All dataframes have been inserted into MongoDB successfully.
